In [41]:
# import
import numpy as np
import cortex
import nibabel as nib
from nipy import load_image
from PIL import Image
import os

# 平均予測脳活動 (セクション毎)

In [44]:
# sub_id_list = [18,22,23,24,28,30,31,35,36,37,38,39,41,42,43,44,45,46,47,48,49,50,51,52,53] # alice all
# sub_id_list = [18,22,28,35,36,37,39,41,42,45,52,58,62,63,64,68,75,76,77,86,87,88,89,91,92,101,106,108,109,114] # male
# sub_id_list = [18,22,28,35,36,37,39,41,42,45,52] # alice male
# sub_id_list = [58,62,63,64,68,75,76,77,86,87,88,89,91,92,101,106,108,109,114] # LPP male
# sub_id_list = [23,24,30,31,38,43,44,46,47,48,49,50,51,53,57,61,62,65,67,69,70,72,73,74,78,79,81,82,83,84,93,94,95,96,97,98,99,100,103,104,105,110,113,115] # female
# sub_id_list = [23,24,30,31,38,43,44,46,47,48,49,50,51,53] # alice female
sub_id_list = [57,61,62,65,67,69,70,72,73,74,78,79,81,82,83,84,93,94,95,96,97,98,99,100,103,104,105,110,113,115] # LPP female
# sub_id_list = [57,58,61,62,63,64,65,67,68,69,70,72,73,74,75,76,77,78,79,81,82,83,84,86,87,
#                88,89,91,92,93,94,95,96,97,98,99,100,101,103,104,105,106,108,109,110,113,114,115] #LPP all

target_subject="sub-22"
section = 9 #Alice の場合は5

roi_image_path = "/home/sync/brain/pycortex_project/resource/Alice/"+target_subject+"/label/"

In [3]:
def min_max_normalization(a, axis=None):
    a_min = a.min(axis=axis, keepdims=True)
    a_max = a.max(axis=axis, keepdims=True)
    return (a - a_min) / (a_max - a_min)

In [ ]:
for i in range(section):
    data_path = "/home/sync/brain/src/all_subject_mean/data/LittlePrince/run_"+str(i+1)+"/"
    # data_path = "/home/sync/brain/src/all_subject_mean/data/Alice/run_"+str(i+1)+"/"
    img_path = "/home/sync/brain/src/all_subject_mean/image/LittlePrince/run_"+str(i+1)+"/"
    # img_path = "/home/sync/brain/src/all_subject_mean/image/Alice/run_"+str(i+1)+"/"
    for j,sub_id in enumerate(sub_id_list):
        if sub_id >= 100:
            subject = "sub_EN"+str(sub_id)
        elif sub_id > 53:
            subject = "sub_EN0"+str(sub_id)
        else:
            subject = "sub-"+str(sub_id)
        new_ver = np.load(data_path+subject+".npy")
        new_ver = min_max_normalization(new_ver)
        dv = cortex.Vertex(new_ver, target_subject, cmap="hot", vmax=new_ver.max(), vmin=new_ver.min())
        fig_sub = cortex.quickflat.make_png(img_path+subject+'.png',dv, with_colorbar=False)

        if j == 0:
            mean_all_subject = new_ver
        else:
            mean_all_subject += new_ver

    mean_all_subject = mean_all_subject / (len(sub_id_list))
    vmax= np.amax(mean_all_subject)
    vmin = np.amin(mean_all_subject)
    # np.save(data_path+"all_subject",mean_all_subject)
    # np.save(data_path+"male_subjects",mean_all_subject)
    np.save(data_path+"female_subjects",mean_all_subject)

    dv_all = cortex.Vertex(mean_all_subject, target_subject, cmap="hot", vmax=vmax, vmin=vmin)
    # fig_all = cortex.quickflat.make_png(img_path+'all_subject_mean.png',dv_all, with_colorbar=False)
    # fig_all = cortex.quickflat.make_png(img_path+'male_subjects_mean.png',dv_all, with_colorbar=False)
    fig_all = cortex.quickflat.make_png(img_path+'female_subjects_mean.png',dv_all, with_colorbar=False)

    brain_img = Image.open(img_path+'female_subjects_mean.png') # 今保存したflatmap
    contour_img = Image.open(roi_image_path+"/roi_contours.png") # 輪郭抽出した画像

    # 重ねて、保存し直します
    brain_img.paste(contour_img, (0, 0), contour_img)
    brain_img.save(img_path+'female_subjects_mean.png')

## 全セクション平均

## Alice

In [23]:
for i in range(5):
    # new_ver = np.load("/home/sync/brain/src/all_subject_mean/data/Alice/run_"+str(i+1)+"/all_subject.npy")
    new_ver = np.load("/home/sync/brain/src/all_subject_mean/data/Alice/run_"+str(i+1)+"/male_subjects.npy")
    # new_ver = np.load("/home/sync/brain/src/all_subject_mean/data/Alice/run_"+str(i+1)+"/female_subjects.npy")
    if i == 0:
        mean_all_subject_all_run = new_ver
    else:
        mean_all_subject_all_run += new_ver

mean_all_subject_all_run = mean_all_subject_all_run / 5
# mean_all_subject_all_run = np.mean(mean_all_subject_all_run,axis=0) # allのときだけon

img_path = "/home/sync/brain/src/all_subject_mean/image/Alice/mean/"
dv_all = cortex.Vertex(mean_all_subject_all_run, target_subject, cmap="hot", vmax=mean_all_subject_all_run.max(), vmin=0)
# fig_all = cortex.quickflat.make_png(img_path+'all_subject_mean.png',dv_all, with_colorbar=False)
fig_all = cortex.quickflat.make_png(img_path+'male_subjects.png',dv_all, with_colorbar=False)
# fig_all = cortex.quickflat.make_png(img_path+'female_subjects.png',dv_all, with_colorbar=False)  

Failed to get connection
** (inkscape:95114): CRITICAL **: 16:49:48.476: dbus_g_proxy_new_for_name: assertion 'connection != NULL' failed
** (inkscape:95114): CRITICAL **: 16:49:48.476: dbus_g_proxy_call: assertion 'DBUS_IS_G_PROXY (proxy)' failed
** (inkscape:95114): CRITICAL **: 16:49:48.476: dbus_g_connection_register_g_object: assertion 'connection != NULL' failed


## LittlePrince

In [ ]:
for i in range(9):
    new_ver = np.load("/home/sync/brain/src/all_subject_mean/data/LittlePrince/run_"+str(i+1)+"/all_subject.npy")
    # new_ver = np.load("/home/sync/brain/src/all_subject_mean/data/LittlePrince/run_"+str(i+1)+"/male_subjects.npy")
    # new_ver = np.load("/home/sync/brain/src/all_subject_mean/data/LittlePrince/run_"+str(i+1)+"/female_subjects.npy")
    if i == 0:
        mean_all_subject_all_run = new_ver
    else:
        mean_all_subject_all_run += new_ver

mean_all_subject_all_run = mean_all_subject_all_run / 9
# mean_all_subject_all_run = np.mean(mean_all_subject_all_run,axis=0) # allのときだけon

img_path = "/home/sync/brain/src/all_subject_mean/image/LittlePrince/mean/"
dv_all = cortex.Vertex(mean_all_subject_all_run, target_subject, cmap="hot", vmax=mean_all_subject_all_run.max(), vmin=0)
fig_all = cortex.quickflat.make_png(img_path+'all_subject_mean.png',dv_all, with_colorbar=False)
# fig_all = cortex.quickflat.make_png(img_path+'male_subjects.png',dv_all, with_colorbar=False)
# fig_all = cortex.quickflat.make_png(img_path+'female_subjects.png',dv_all, with_colorbar=False)  

# 女性被験者平均 - 男性被験者平均

## Alice

In [48]:
for i in range(5):
    # new_ver = np.load("/home/sync/brain/src/all_subject_mean/data/Alice/run_"+str(i+1)+"/all_subject.npy")
    new_ver_m = np.load("/home/sync/brain/src/all_subject_mean/data/Alice/run_"+str(i+1)+"/male_subjects.npy")
    new_ver_f = np.load("/home/sync/brain/src/all_subject_mean/data/Alice/run_"+str(i+1)+"/female_subjects.npy")
    if i == 0:
        mean_all_subject_all_run_m = new_ver_m
        mean_all_subject_all_run_f = new_ver_f
    else:
        mean_all_subject_all_run_m += new_ver_m
        mean_all_subject_all_run_f += new_ver_f

mean_all_subject_all_run_m = mean_all_subject_all_run_m / 5
mean_all_subject_all_run_f = mean_all_subject_all_run_f / 5
diff = mean_all_subject_all_run_f - mean_all_subject_all_run_m

img_path = "/home/sync/brain/src/all_subject_mean/image/Alice/mean/"
dv_all = cortex.Vertex(diff, target_subject, cmap="bwr", vmax=mean_all_subject_all_run.max(), vmin=-mean_all_subject_all_run.max())
fig_all = cortex.quickflat.make_png(img_path+'f-m.png',dv_all, with_colorbar=False)

brain_img = Image.open(img_path+'f-m.png') # 保存したflatmap
contour_img = Image.open(roi_image_path+"/roi_contours.png") # 輪郭抽出した画像

# 重ねて、保存し直します
brain_img.paste(contour_img, (0, 0), contour_img)
brain_img.save(img_path+'f-m.png')

Failed to get connection
** (inkscape:65597): CRITICAL **: 01:34:53.270: dbus_g_proxy_new_for_name: assertion 'connection != NULL' failed
** (inkscape:65597): CRITICAL **: 01:34:53.270: dbus_g_proxy_call: assertion 'DBUS_IS_G_PROXY (proxy)' failed
** (inkscape:65597): CRITICAL **: 01:34:53.270: dbus_g_connection_register_g_object: assertion 'connection != NULL' failed


## LittlePrince

In [49]:
for i in range(9):
    # new_ver = np.load("/home/sync/brain/src/all_subject_mean/data/Alice/run_"+str(i+1)+"/all_subject.npy")
    new_ver_m = np.load("/home/sync/brain/src/all_subject_mean/data/LittlePrince/run_"+str(i+1)+"/male_subjects.npy")
    new_ver_f = np.load("/home/sync/brain/src/all_subject_mean/data/LittlePrince/run_"+str(i+1)+"/female_subjects.npy")
    if i == 0:
        mean_all_subject_all_run_m = new_ver_m
        mean_all_subject_all_run_f = new_ver_f
    else:
        mean_all_subject_all_run_m += new_ver_m
        mean_all_subject_all_run_f += new_ver_f

mean_all_subject_all_run_m = mean_all_subject_all_run_m / 5
mean_all_subject_all_run_f = mean_all_subject_all_run_f / 5
diff = mean_all_subject_all_run_f - mean_all_subject_all_run_m

img_path = "/home/sync/brain/src/all_subject_mean/image/LittlePrince/mean/"
dv_all = cortex.Vertex(diff, target_subject, cmap="bwr", vmax=mean_all_subject_all_run.max(), vmin=-mean_all_subject_all_run.max())
fig_all = cortex.quickflat.make_png(img_path+'f-m.png',dv_all, with_colorbar=False)

brain_img = Image.open(img_path+'f-m.png') # 保存したflatmap
contour_img = Image.open(roi_image_path+"/roi_contours.png") # 輪郭抽出した画像

# 重ねて、保存し直します
brain_img.paste(contour_img, (0, 0), contour_img)
brain_img.save(img_path+'f-m.png')

Failed to get connection
** (inkscape:65727): CRITICAL **: 01:35:08.445: dbus_g_proxy_new_for_name: assertion 'connection != NULL' failed
** (inkscape:65727): CRITICAL **: 01:35:08.445: dbus_g_proxy_call: assertion 'DBUS_IS_G_PROXY (proxy)' failed
** (inkscape:65727): CRITICAL **: 01:35:08.445: dbus_g_connection_register_g_object: assertion 'connection != NULL' failed
